create 1 GMM per TIMIT phone type, fitted only on those phone observations.

In [1]:
# general imports
import os
import pickle
import numpy as np
import copy

In [2]:
# import Scykit learn GMM library
from sklearn import mixture

In [3]:
# import custom functions
import sys
# path to libraries
# currently in ../scripts-lib/
tool_path = os.path.abspath('..' + os.sep + 'scripts-lib')

if tool_path not in sys.path:
    sys.path.append(tool_path)
import lib_phones as lph

# print the loaded functions
print dir(lph)[5:]

['find_phone_index', 'load_phone_file']


In [4]:
# load phone list
phone_path = os.path.abspath('../datasets/TIMIT-MFCCs/TIMIT_phone_list.txt')
phone_list = lph.load_phone_file(phone_path)
print len(phone_list), phone_list

61 ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']


In [5]:
#load mfccs into sklearn observations, each frame is an obs

train_TIMIT_dir = os.path.abspath('../datasets/TIMIT-MFCCs/dev')

#create individual obs list for each phone type
train_phone_obs_dict = {}
for phone in phone_list:
    train_phone_obs_dict[phone] = []

# complete obs 
train_obs = []
train_obs_labels = []

# walk the directories
for (path, dirs, files) in os.walk(train_TIMIT_dir):
    print "working in path : " + path

    for file in files:
        # skip the SA files
        #dev, only work on file si1573.mfcc.csv     "si1573" in file and
        if ".mfcc" in file  and "sa" not in file:
            #check if corresponding .phn file exists
            if not os.path.exists(path + "/" + file[:-8] + "phn"):
                print path + "/" + file[:-8] + "phn"
                print "corresponding .phn file does not exist!"
            else:
                
                print "working on: " + file
#                 print "from path : " + path

                # open the files
                mfcc_file = open(path + "/" + file)
                phn_file = open(path + "/" + file[:-8] + "phn")

                # extract phone times
                phone_times = []
                for phn_line in phn_file:
                    phone_times.append(phn_line.split())
                # transpose for easier use
                phone_times = map(list, zip(*phone_times))

                # skip mfcc_file header
                next(mfcc_file)

                # reset frame count
                frame_cnt = 0

                # for each line of mfcc_file
                for mfcc_line in mfcc_file:

                    # increment frame count
                    frame_cnt += 1 

                    # print "frame line #:", frame_cnt 

                    # frame start time in seconds
                    start_t = mfcc_line.split(";")[1]

                    # create frame (skiping first 2 values, frame_index and frame_time)
                    frame = map( float,  mfcc_line.split(";")[2:])
                    # print numpy.shape(frame)
                    # print frame

                    # find correspond phoneme and index in the list
                    phn_index = lph.find_phone_index(start_t, phone_times, phone_list)

                    # add to instances for corresponding 
                    train_phone_obs_dict[phone_list[phn_index]].append(frame)
                    # add to instances
                    train_obs.append(frame)
                    train_obs_labels.append(phone_list[phn_index])

working in path : C:\Users\FG\Desktop\PhD\Research\Reservoirs\datasets\TIMIT-MFCCs\dev
working on: si1027.mfcc.csv
working on: si1105.mfcc.csv
working on: si1657.mfcc.csv
working on: si1735.mfcc.csv
working on: si475.mfcc.csv
working on: si648.mfcc.csv
working on: sx115.mfcc.csv
working on: sx127.mfcc.csv
working on: sx205.mfcc.csv
working on: sx217.mfcc.csv
working on: sx25.mfcc.csv
working on: sx295.mfcc.csv
working on: sx307.mfcc.csv
working on: sx37.mfcc.csv
working on: sx385.mfcc.csv
working on: sx397.mfcc.csv


In [6]:
# smallest phone observations
min([len(train_phone_obs_dict[phone]) for phone in phone_list])

0

In [7]:
# gmm parameters
num_components = 10
num_iter=2

# dictionary containing a gmm for each phone, trained directly on obervation on those phones
gmm_dict = dict()

# for each phone, build a GMM
for phone in phone_list:
    
    # check if enough observations exist
    if len(train_phone_obs_dict[phone]) < num_components:
        print "not enough obs for phone", phone
        pass
    else:
        print "training gmm for phone", phone
        g = mixture.GMM(n_components= num_components, n_iter=num_iter) 
        g.fit(train_phone_obs_dict[phone])
        gmm_dict[phone] = g
    

training gmm for phone aa
training gmm for phone ae
training gmm for phone ah
training gmm for phone ao
training gmm for phone aw
training gmm for phone ax
training gmm for phone ax-h
training gmm for phone axr
training gmm for phone ay
training gmm for phone b
training gmm for phone bcl
not enough obs for phone ch
training gmm for phone d
training gmm for phone dcl
training gmm for phone dh
training gmm for phone dx
training gmm for phone eh
training gmm for phone el
training gmm for phone em
not enough obs for phone en
not enough obs for phone eng
training gmm for phone epi
training gmm for phone er
training gmm for phone ey
training gmm for phone f
training gmm for phone g
training gmm for phone gcl
training gmm for phone h#
training gmm for phone hh
training gmm for phone hv
training gmm for phone ih
training gmm for phone ix
training gmm for phone iy
training gmm for phone jh
training gmm for phone k
training gmm for phone kcl
training gmm for phone l
training gmm for phone m
trai

In [8]:
#save gmm in pickled form
import cPickle as pickle

# name and location to save in
pickle_name = "TIMIT_gmm_unadapted_dict" + ".pckl"
pickle_dir = os.path.abspath('..'+ os.sep + 'datasets' + os.sep + 'TIMIT Pickled Data')

if not os.path.isdir(pickle_dir):
    os.makedirs(pickle_dir)
    
pickle.dump( gmm_dict, open( pickle_dir + os.sep + pickle_name, "wb") )
print "saved unadapted gmm dictionary in ", pickle_dir + os.sep + pickle_name

saved unadapted gmm dictionary in  C:\Users\FG\Desktop\PhD\Research\Reservoirs\datasets\TIMIT Pickled Data\TIMIT_gmm_unadapted_dict.pckl
